In [1]:
import random
import os
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
import _pickle as cPickle
import pickle
with open('CODE_POST_OVERALL-EMBEDDINGS_DATA_V3.pkl', 'rb') as f:
    OVO_data = cPickle.load(f)
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

/home/21fl11/.local/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [3]:
raw_accepted_answers = [item['raw_accepted_answer'] for item in OVO_data]

# answer_embeddings_list = [item['answer_embeddings'] for item in OVO_data]

answer_sentences_list = [item['answer_sentences'] for item in OVO_data]

In [4]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import torch
import json
from tqdm import tqdm
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import nltk
from nltk.tokenize import sent_tokenize

# Download the Punkt Tokenizer Models (only need to do this once)
nltk.download('punkt')

/home/21fl11/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /home/21fl11/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
device = "cuda"
encoder_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2').to(device)

In [6]:
all_sentences = [sentence for group in answer_sentences_list for sentence in group]

In [ ]:
import numpy as np
save_dir = "AnswerEmbedding"
os.makedirs(save_dir, exist_ok=True)

# Number of parts to split into
num_parts = 4
part_size = len(all_sentences) // num_parts

for part_index in range(num_parts):
    start = part_index * part_size
    end = (part_index + 1) * part_size if part_index < num_parts - 1 else len(all_sentences)
    sentences_chunk = all_sentences[start:end]

    # Encode this chunk in manageable batches
    batch_size = 32
    chunk_embeddings_list = []

    for i in tqdm(range(0, len(sentences_chunk), batch_size), desc=f"Encoding part {part_index+1}/{num_parts}"):
        batch = sentences_chunk[i:i+batch_size]
        batch_embeddings = encoder_model.encode(
            batch,
            convert_to_tensor=True,
            device=device,
            show_progress_bar=False
        )
        # Move to CPU and convert to numpy to free GPU memory
        batch_embeddings = batch_embeddings.cpu().numpy()
        chunk_embeddings_list.append(batch_embeddings)

    # Concatenate all embeddings for this chunk
    chunk_embeddings = np.concatenate(chunk_embeddings_list, axis=0)
    # Save this chunk's embeddings
    np.save(os.path.join(save_dir, f"embeddings_part_{part_index+1}.npy"), chunk_embeddings)
    print(f"Saved embeddings for part {part_index+1} with shape {chunk_embeddings.shape}")

print("All parts processed and saved.")

Encoding part 1/4: 100%|██████████| 124921/124921 [1:07:11<00:00, 30.98it/s]


Saved embeddings for part 1 with shape (3997446, 768)


Encoding part 2/4: 100%|██████████| 124921/124921 [1:46:23<00:00, 19.57it/s]  


Saved embeddings for part 2 with shape (3997446, 768)


Encoding part 3/4: 100%|██████████| 124921/124921 [2:39:52<00:00, 13.02it/s]  


Saved embeddings for part 3 with shape (3997446, 768)


Encoding part 4/4: 100%|██████████| 124921/124921 [2:39:31<00:00, 13.05it/s]  


Saved embeddings for part 4 with shape (3997446, 768)
All parts processed and saved.


In [9]:
import numpy as np

# Specify the path to your .npy file
filename = "AnswerEmbedding/embeddings_part_1.npy"

# Load the npy file into a numpy array
embeddings = np.load(filename)

# Now 'embeddings' is a NumPy array containing the data from the file
print("Loaded embeddings shape:", embeddings.shape)
print("First embedding:", embeddings[0])

Loaded embeddings shape: (3997446, 768)
First embedding: [-1.59766153e-02 -2.21100752e-03 -1.44797582e-02 -1.44546721e-02
 -2.25519501e-02 -7.97338842e-04  1.63116585e-02  3.64094228e-02
 -2.75785495e-02  3.35127255e-03  2.64206752e-02 -2.23280452e-02
  2.26558503e-02  1.69685688e-02  1.05268862e-02 -2.23046169e-02
 -1.13898451e-02 -2.26948317e-03 -6.96935058e-02 -3.35645527e-02
 -8.67899321e-03  1.12106334e-02 -1.34580946e-02  2.57405434e-02
  2.57606735e-03 -2.59166211e-02 -3.71751562e-02 -3.53817232e-02
 -1.76877063e-02 -2.37585120e-02  3.63303674e-03  2.13548206e-02
 -2.24412140e-02 -1.52949197e-02  2.17884735e-06 -1.84437130e-02
  1.75382141e-02 -9.97955631e-03  1.50554134e-02  6.02692813e-02
 -1.50444778e-02 -1.33065169e-03 -5.64483851e-02  6.69354200e-03
 -4.80605103e-03 -5.32964095e-02  4.13686559e-02 -7.74481334e-03
  2.83558611e-02 -4.59222682e-02  2.75945868e-02 -4.84167188e-02
 -1.80807337e-02 -1.00901276e-02  8.77488032e-02  2.97218561e-02
 -1.65045969e-02  4.46206424e-03 

In [10]:
np.save("AnswerEmbedding/all_sentences.npy", np.array(all_sentences, dtype=object))
print("Saved all_sentences with length:", len(all_sentences))

Saved all_sentences with length: 15989784


In [11]:
emb_part_1 = np.load("AnswerEmbedding/embeddings_part_1.npy")
emb_part_2 = np.load("AnswerEmbedding/embeddings_part_2.npy")
emb_part_3 = np.load("AnswerEmbedding/embeddings_part_3.npy")
emb_part_4 = np.load("AnswerEmbedding/embeddings_part_4.npy")

In [12]:
all_embeddings = np.concatenate([emb_part_1, emb_part_2, emb_part_3, emb_part_4], axis=0)

In [14]:
len(all_embeddings)

15989784

In [13]:
# Load the just-saved all_sentences array to verify alignment
loaded_sentences = np.load("AnswerEmbedding/all_sentences.npy", allow_pickle=True)

# Ensure lengths match
assert len(loaded_sentences) == all_embeddings.shape[0], "Mismatch in number of sentences and embeddings!"


In [15]:
np.save("AnswerEmbedding/all_embeddings.npy", all_embeddings)

print("All embeddings combined and saved. Shape:", all_embeddings.shape)

All embeddings combined and saved. Shape: (15989784, 768)
